<a href="https://colab.research.google.com/github/azzindani/00_Data_Source/blob/main/00_Llama3_2_3B_Fine_Tune_PEFT_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 00 Import Modules

In [ ]:
#!pip install --upgrade transformers
!pip install -q peft
!pip install -U -q bitsandbytes
!pip install -q datasets
!pip install -q trl

In [ ]:
import os
import pathlib
import torch
import numpy as np

from datetime import datetime
from datasets import load_dataset
from datasets import Dataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from trl import SFTTrainer

from transformers import (
  AutoTokenizer,
  AutoModelForCausalLM,
  AutoModelForSeq2SeqLM,
  AutoModel,
  AutoModelForSequenceClassification,
  DataCollatorForLanguageModeling,
  Trainer,
  TrainingArguments,
  pipeline,
  TextDataset,
  EvalPrediction,
  DataCollatorWithPadding,
  GenerationConfig,
  BitsAndBytesConfig,
  DataCollatorForSeq2Seq,
  TextStreamer
)

from peft import (
  LoraConfig,
  PeftModelForSequenceClassification,
  PeftModel,
  TaskType,
  AutoPeftModelForSequenceClassification,
  get_peft_model,
  prepare_model_for_kbit_training
)

if torch.cuda.is_available():
  print("GPU is available!")
else:
  print("GPU is not available.")

GPU is available!


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

## 01 Import Model

In [ ]:
#url = 'https://huggingface.co/Qwen/Qwen2.5-0.5B'
#model_name = url.split('.co/')[-1]

model_name = 'unsloth/Meta-Llama-3.1-8B-Instruct'

In [ ]:
bnb_config = BitsAndBytesConfig(
  load_in_4bit = True,
  bnb_4bit_quant_type = 'nf4',
  bnb_4bit_compute_dtype = torch.float16,
  bnb_4bit_use_double_quant = True,
)

model = AutoModelForCausalLM.from_pretrained(
  model_name,
  quantization_config = bnb_config,
  trust_remote_code = True
).to(device) #'''

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/969 [00:00<?, ?B/s]

`low_cpu_mem_usage` was None, now default to True since model is quantized.


model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/234 [00:00<?, ?B/s]

In [ ]:
'''model = AutoModelForCausalLM.from_pretrained(
  model_name,
  torch_dtype = torch.float16,
  trust_remote_code = True
).to(device) #'''

'model = AutoModelForCausalLM.from_pretrained(\n  model_name,\n  torch_dtype = torch.float16,\n  trust_remote_code = True\n).to(device) #'

In [ ]:
model

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096, padding_idx=128004)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): Llama

In [ ]:
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
trainable_percentage = (trainable_params / total_params) * 100

print('Total parameters :', total_params)
print('Trainable parameters :', trainable_params)
print('Trainable percentage: {:.2f}%'.format(trainable_percentage))

Total parameters : 4540600320
Trainable parameters : 1050939392
Trainable percentage: 23.15%


## 02 Import Tokenizer

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
#tokenizer

tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/340 [00:00<?, ?B/s]

## 03 Pre Test

In [ ]:
def assistant(prompt):
  messages = [
    {'role' : 'human', 'content' : prompt},
  ]
  inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True,
    return_tensors = 'pt',
  ).to('cuda')
  generation_config = GenerationConfig(
    do_sample = True,
    top_k = 1,
    temperature = 0.1,
    max_new_tokens = 1024,
    pad_token_id = tokenizer.eos_token_id
  )
  outputs = model.generate(
    input_ids = inputs,
    generation_config = generation_config
  )
  return print(tokenizer.decode(outputs[0]))#, skip_special_tokens = True))

In [ ]:
prompt = '''
Summarize the following legal text in a few sentences:

'In the case of John Doe v. XYZ Corp, the plaintiff, John Doe, entered into a formal service contract with XYZ Corp in January 2022. The agreement stipulated a one-year commitment for IT support services, with John Doe providing on-site troubleshooting, software updates, and system maintenance. XYZ Corp agreed to pay a fixed monthly retainer along with additional fees for after-hours support. However, in June 2022, XYZ Corp terminated the contract without prior notice, claiming that an unexpected downturn in business operations left them financially unable to continue. The plaintiff alleges wrongful termination, asserting that XYZ Corp failed to adhere to the 60-day notice clause outlined in the contract. Additionally, the plaintiff contends that the early termination damaged his professional reputation and resulted in significant financial losses, including missed client opportunities and incurred expenses for certifications specific to XYZ Corp’s systems. John Doe is seeking compensation for the remaining contract balance, damages for reputational harm, and reimbursement for training and certification costs required under the agreement.
'''
assistant(prompt)

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

<|eot_id|><|start_header_id|>human<|end_header_id|>

Summarize the following legal text in a few sentences:

'In the case of John Doe v. XYZ Corp, the plaintiff, John Doe, entered into a formal service contract with XYZ Corp in January 2022. The agreement stipulated a one-year commitment for IT support services, with John Doe providing on-site troubleshooting, software updates, and system maintenance. XYZ Corp agreed to pay a fixed monthly retainer along with additional fees for after-hours support. However, in June 2022, XYZ Corp terminated the contract without prior notice, claiming that an unexpected downturn in business operations left them financially unable to continue. The plaintiff alleges wrongful termination, asserting that XYZ Corp failed to adhere to the 60-day notice clause outlined in the contract. Additionally, the plaintiff contends that the early 

In [ ]:
prompt = "What are the legal implications if a party violates a confidentiality agreement in the context of contract law? For example, consider a scenario where a contractor working with Tech Innovators Inc. shares proprietary technology information with a competitor. Explain in detail and cite relevant case law where possible."
assistant(prompt)

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

<|eot_id|><|start_header_id|>human<|end_header_id|>

What are the legal implications if a party violates a confidentiality agreement in the context of contract law? For example, consider a scenario where a contractor working with Tech Innovators Inc. shares proprietary technology information with a competitor. Explain in detail and cite relevant case law where possible.<|eot_id|><|start_header_id|>assistant<|end_header_id|>

In the context of contract law, a confidentiality agreement (also known as a non-disclosure agreement or NDA) is a legally binding contract between two parties that outlines the terms and conditions under which one party (the "disclosing party") shares confidential information with the other party (the "receiving party"). If the receiving party violates the confidentiality agreement by disclosing the confidential information to a third party, 

In [ ]:
prompt = "Rephrase the following legal statement to make it more understandable for a general audience: 'Under the terms of the non-compete agreement, the defendant is barred from engaging in any business that competes with the plaintiff's business within a 50-mile radius for two years following the termination of employment.' Retain all key information while simplifying the language."
assistant(prompt)

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

<|eot_id|><|start_header_id|>human<|end_header_id|>

Rephrase the following legal statement to make it more understandable for a general audience: 'Under the terms of the non-compete agreement, the defendant is barred from engaging in any business that competes with the plaintiff's business within a 50-mile radius for two years following the termination of employment.' Retain all key information while simplifying the language.<|eot_id|><|start_header_id|>assistant<|end_header_id|>

Here's a rephrased version of the statement:

"After leaving their job, the defendant is not allowed to start or work for any business that directly competes with the plaintiff's business within a 50-mile radius for two years. This means they cannot open a similar store, start a competing company, or work for a business that offers the same products or services as the plaintiff's busine

In [ ]:
prompt = "In the case where a defendant claims breach of contract due to unforeseeable events, how does the principle of 'force majeure' apply? For instance, if a company was unable to deliver contracted goods due to a natural disaster, provide a detailed explanation and outline any relevant conditions under which the force majeure principle might or might not be applicable."
assistant(prompt)

In [ ]:
prompt = "Construct an argument in defense of a client accused of breaching intellectual property laws due to sharing copyrighted material in an educational setting. For example, a teacher shares portions of a textbook with students to support classroom discussion. Focus on any legal exceptions or defenses that may apply, such as the fair use doctrine in educational contexts."
assistant(prompt)

## 04 Import Dataset

In [ ]:
#url = 'https://huggingface.co/datasets/KingNish/reasoning-base-20k'
#dataset_name = url.split('datasets/')[-1]

dataset_name = 'mlabonne/FineTome-100k'

In [ ]:
max_length = 1024

In [ ]:
dataset = load_dataset(dataset_name, split = 'train')
dataset

In [ ]:
dataset.select(range(5)).to_pandas().head()

In [ ]:
dataset[0]

In [ ]:
features = list(dataset.features.keys())
print(features)

## 05 Text Formatting

In [ ]:
def transform_conversations(example):
  role_map = {
    'human' : 'user',
    'gpt' : 'assistant'
  }
  transformed_conversations = [
    {
      'role' : role_map.get(turn['from'], turn['from']),
      'content' : turn['value']
    }
    for turn in example['conversations']
  ]
  return {'conversations': transformed_conversations}

In [ ]:
formatted_dataset = dataset.map(transform_conversations, remove_columns = features)
formatted_dataset

In [ ]:
print(formatted_dataset[0]['conversations'])

In [ ]:
def format_conversation(example):
  for entry in example['conversations']:
    role = entry['role']
    content = entry['content']

    if role == 'user':
      formatted_text = f"<|start_header_id|>{role}<|end_header_id|>\n\n{content}\n<|eot_id|>"
    elif role == 'assistant':
      formatted_text += f"<|start_header_id|>{role}<|end_header_id|>\n\n{content}\n<|eot_id|>"

  return {'prompt': formatted_text}

In [ ]:
formatted_dataset = formatted_dataset.map(
  format_conversation,
  remove_columns = list(formatted_dataset.features.keys())
)
formatted_dataset

In [ ]:
print(formatted_dataset[0]['prompt'])

## 06 Tokenization

In [ ]:
def tokenize_data(example, max_length = max_length):
  return tokenizer(example['prompt'], truncation = True, padding = 'max_length', max_length = max_length)

In [ ]:
tokenized_dataset = formatted_dataset.map(tokenize_data, batched = True)#, remove_columns = 'text')
tokenized_dataset

In [ ]:
print(tokenized_dataset[0]['prompt'])

In [ ]:
dataset = tokenized_dataset.train_test_split(test_size = 0.1, seed = 42)
dataset

In [ ]:
train_dataset = dataset['train']
test_dataset = dataset['test']
train_dataset

In [ ]:
train_dataset.select(range(5)).to_pandas().head()

In [ ]:
print(train_dataset[0]['prompt'])

In [ ]:
print(train_dataset[0]['input_ids'])

In [ ]:
print(train_dataset[0]['attention_mask'])

## 07 Data Collator Set Up

In [ ]:
#data_collator = DataCollatorWithPadding(tokenizer = tokenizer)
#data_collator = DataCollatorForSeq2Seq(tokenizer = tokenizer)
data_collator = DataCollatorForLanguageModeling(tokenizer = tokenizer, mlm = False)

## 08 Evaluation Metrics Set Up

In [ ]:
def compute_metrics(p: EvalPrediction):
  preds = np.argmax(p.predictions, axis = 1)
  precision, recall, f1, _ = precision_recall_fscore_support(
    p.label_ids,
    preds,
    average = 'weighted'
  )
  matrix = {
    'accuracy': accuracy_score(p.label_ids, preds),
    'f1': f1, 'precision': precision,
    'recall': recall
  }
  return matrix

In [ ]:
torch.cuda.empty_cache()

## 09 Set Up PEFT / LoRA / QLoRA

In [ ]:
lora_alpha = 16
lora_dropout = 0.1
lora_r = 64
target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                  "gate_proj", "up_proj", "down_proj",]
peft_config = LoraConfig(
  lora_alpha = lora_alpha,
  lora_dropout = lora_dropout,
  r = lora_r,
  bias = 'none',
  task_type = 'CAUSAL_LM',
  target_modules = target_modules,
)

In [ ]:
peft_model = get_peft_model(model, peft_config, adapter_name = 'math')
peft_model.print_trainable_parameters()

## 10 Training Model

In [ ]:
model

In [ ]:
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
trainable_percentage = (trainable_params / total_params) * 100

print('Total parameters :', total_params)
print('Trainable parameters :', trainable_params)
print('Trainable percentage: {:.2f}%'.format(trainable_percentage))

In [ ]:
torch.cuda.empty_cache()

In [ ]:
save_path = './model'

batch_size = 1
max_steps = 200
training_args = TrainingArguments(
  output_dir = save_path,
  gradient_accumulation_steps = 2,
  evaluation_strategy = 'steps',
  do_eval = True,
  per_device_train_batch_size = batch_size,
  per_device_eval_batch_size = 2,
  log_level = 'debug',
  save_strategy = 'no',
  save_total_limit = 2,
  save_safetensors = False,
  fp16 = True,
  logging_steps = 20,
  learning_rate = 2e-5,
  eval_steps = 20,
  max_steps = max_steps,
  warmup_steps = 30,
  lr_scheduler_type = 'cosine',
)
training_args

In [ ]:
trainer = SFTTrainer(
  model = model,
  train_dataset = train_dataset,#.select(range(10000)),
  eval_dataset = test_dataset.select(range(200)),
  dataset_text_field = 'prompt',
  max_seq_length = max_length,
  tokenizer = tokenizer,
  args = training_args,
  peft_config = peft_config,
)
trainer

In [ ]:
trainer.train()

## 11 Model Evaluation

In [ ]:
evaluation_results = trainer.evaluate()
print('Evaluation Results:', evaluation_results)

## 12 Save Model

In [ ]:
save_model = trainer.model.module if hasattr(trainer.model, 'module') else trainer.model
save_model.save_pretrained(save_path)

## 13 Load PEFT Model

In [ ]:
torch.cuda.empty_cache()

In [ ]:
peft_model = PeftModel.from_pretrained(model, save_path)
peft_model

In [ ]:
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
trainable_percentage = (trainable_params / total_params) * 100

print('Total parameters :', total_params)
print('Trainable parameters :', trainable_params)
print('Trainable percentage: {:.2f}%'.format(trainable_percentage))

## 14 Post Test

In [ ]:
def assistant(prompt):
  messages = [
    {'role' : 'human', 'content' : prompt},
  ]
  inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True,
    return_tensors = 'pt',
  ).to('cuda')
  generation_config = GenerationConfig(
    do_sample = True,
    top_k = 1,
    temperature = 0.1,
    max_new_tokens = 1024,
    pad_token_id = tokenizer.eos_token_id
  )
  outputs = peft_model.generate(
    input_ids = inputs,
    generation_config = generation_config
  )
  return print(tokenizer.decode(outputs[0]))#, skip_special_tokens = True))

In [ ]:
prompt = '''
Summarize the following legal text in a few sentences:

'In the case of John Doe v. XYZ Corp, the plaintiff, John Doe, entered into a formal service contract with XYZ Corp in January 2022. The agreement stipulated a one-year commitment for IT support services, with John Doe providing on-site troubleshooting, software updates, and system maintenance. XYZ Corp agreed to pay a fixed monthly retainer along with additional fees for after-hours support. However, in June 2022, XYZ Corp terminated the contract without prior notice, claiming that an unexpected downturn in business operations left them financially unable to continue. The plaintiff alleges wrongful termination, asserting that XYZ Corp failed to adhere to the 60-day notice clause outlined in the contract. Additionally, the plaintiff contends that the early termination damaged his professional reputation and resulted in significant financial losses, including missed client opportunities and incurred expenses for certifications specific to XYZ Corp’s systems. John Doe is seeking compensation for the remaining contract balance, damages for reputational harm, and reimbursement for training and certification costs required under the agreement.
'''
assistant(prompt)

In [ ]:
prompt = "What are the legal implications if a party violates a confidentiality agreement in the context of contract law? For example, consider a scenario where a contractor working with Tech Innovators Inc. shares proprietary technology information with a competitor. Explain in detail and cite relevant case law where possible."
assistant(prompt)

In [ ]:
prompt = "Rephrase the following legal statement to make it more understandable for a general audience: 'Under the terms of the non-compete agreement, the defendant is barred from engaging in any business that competes with the plaintiff's business within a 50-mile radius for two years following the termination of employment.' Retain all key information while simplifying the language."
assistant(prompt)

In [ ]:
prompt = "In the case where a defendant claims breach of contract due to unforeseeable events, how does the principle of 'force majeure' apply? For instance, if a company was unable to deliver contracted goods due to a natural disaster, provide a detailed explanation and outline any relevant conditions under which the force majeure principle might or might not be applicable."
assistant(prompt)

In [ ]:
prompt = "Construct an argument in defense of a client accused of breaching intellectual property laws due to sharing copyrighted material in an educational setting. For example, a teacher shares portions of a textbook with students to support classroom discussion. Focus on any legal exceptions or defenses that may apply, such as the fair use doctrine in educational contexts."
assistant(prompt)